In [ ]:
import os

try:
    # Running as normal Python script inside src/
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)                        # EMOTION-PRED/src
    project_root = os.path.dirname(src_root)                    # EMOTION-PRED/
except NameError:
    # Running inside Jupyter (likely src/notebooks or src/)
    cwd = os.getcwd()

    # If running inside src/notebooks → go up one level
    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)
    else:
        # Running from project root directly
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# Final unified paths
results_root = os.path.join(src_root, "results")
data_root = os.path.join(src_root, "data","MAMS-ACSA","raw","data_jsonl")
print(f"📂 Project root: {project_root}"
      f"\n📂 Source root: {src_root}"
      f"\n📂 Results root: {results_root}"
      f"\n📂 Data root: {data_root}")
# 3 — JSONL files
TRAIN_JSONL = os.path.join(data_root, "train.jsonl")
VAL_JSONL   = os.path.join(data_root, "val.jsonl")
TEST_JSONL  = os.path.join(data_root, "test.jsonl")
SAMPLE_JSONL = os.path.join(data_root, "sample.jsonl")
print("Using dataset directory:", data_root)



In [ ]:
# import json
# import os
# import requests
# from dotenv import load_dotenv

# # -----------------------------
# # Load API key
# # -----------------------------
# load_dotenv()
# API_KEY = os.getenv("GEMINI_API_KEY")

# MODEL = "models/gemini-2.5-flash"
# URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"

# HEADERS = {
#     "Content-Type": "application/json",
#     "X-goog-api-key": API_KEY
# }

# # -----------------------------
# # Paths
# # -----------------------------
# IN_PATH = os.path.join(data_root, "sample_06_12_2025_6pm_annotated.jsonl")
# EMOTION_JSON = os.path.join(data_root, "emotion.json")

# OUT_DIR = "output"
# OUT_PATH = os.path.join(OUT_DIR, "gemini_annotated.jsonl")
# os.makedirs(OUT_DIR, exist_ok=True)

# # -----------------------------
# # Load data + emotion lists
# # -----------------------------
# data = [json.loads(line) for line in open(IN_PATH, "r", encoding="utf-8")]
# EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))

# # Build lookup table
# allowed_lookup = {
#     (aspect, polarity): set(EMOTIONS[aspect][polarity])
#     for aspect in EMOTIONS
#     for polarity in EMOTIONS[aspect]
# }

# # -----------------------------
# # Ask Gemini helper
# # -----------------------------
# def ask_gemini(prompt):
#     payload = {
#         "contents": [{"parts": [{"text": prompt}]}]
#     }
#     r = requests.post(URL, headers=HEADERS, json=payload)
#     r.raise_for_status()
#     return r.json()["candidates"][0]["content"]["parts"][0]["text"].strip()

# # -----------------------------
# # Get emotion
# # -----------------------------
# def get_emotion(review, aspect, polarity):
#     allowed = allowed_lookup[(aspect, polarity)]

#     prompt = f"""
# Choose the emotion toward the given aspect.

# Allowed emotions: {list(allowed)}

# Rules:
# - Respond with EXACTLY ONE WORD from the allowed emotions.
# - Do NOT invent new emotions.
# - Do NOT output sentiment words.

# Review: "{review}"
# Aspect: "{aspect}"
# Polarity: "{polarity}"

# Return ONLY the emotion word.
# """

#     resp = ask_gemini(prompt)
#     resp = resp.replace(".", "").replace(",", "").title()

#     if resp in allowed:
#         return resp
#     else:
#         # fallback: first allowed emotion
#         return list(allowed)[0]

# # -----------------------------
# # Annotate file
# # -----------------------------
# for row in data:
#     text = row["input"]
#     for item in row["output"]:
#         item["emotion"] = get_emotion(text, item["aspect"], item["polarity"])

# # -----------------------------
# # Save output
# # -----------------------------
# with open(OUT_PATH, "w", encoding="utf-8") as f:
#     for row in data:
#         f.write(json.dumps(row, ensure_ascii=False) + "\n")

# print("DONE →", OUT_PATH)

In [ ]:
GUIDELINES = """
ASPECT DEFINITIONS & CUES
-----------------------------------------
AMBIENCE (Atmosphere, Environment)
• Physical atmosphere or sensory environment: decor, lighting, music/noise, cleanliness, smell, comfort, temperature, seating, overall vibe.
• Common cues: “vibe”, “atmosphere”, “environment”, “setting”, “decor”, “layout”, “cozy”, “romantic”, “noisy”, “crowded”, “dirty”.
• Positive: lovely atmosphere, cozy.
• Negative: too loud, dirty tables.
• Neutral: dimly lit.

FOOD
• Taste, flavor, appearance, freshness, quality of dishes.
• Cues: “taste”, “flavor”, “freshness”, “undercooked”, “overcooked”, “delicious”, “gross”.
• Positive: delicious pizza.
• Negative: dry meat, awful soup.
• Neutral: factual statements about dishes.

MENU
• Menu size, variety, dietary options, availability, menu layout, confusing menus.
• Cues: “selection”, “options”, “variety”, “vegan options”, “limited”, “menu unavailable”, “confusing”.
• Positive: great variety.
• Negative: unavailable items, confusing layout.
• Neutral: normal children’s menu.

PLACE (Location)
• Location, neighborhood safety, accessibility, parking, ease of finding.
• Cues: “location”, “area”, “parking”, “hard to find”, “near station”.
• Positive: perfect location.
• Negative: unsafe area, impossible parking.
• Neutral: located next to station.

PRICE (Cost, Value for Money)
• Perception of price fairness, expensive/cheap, value for quality.
• Cues: “expensive”, “overpriced”, “cheap”, “worth it”, “value”, “rip-off”.
• Positive: good prices for portions.
• Negative: overpriced, not worth the price.
• Neutral: menu costs X.

SERVICE (Process Efficiency, Speed, Organization)
• Operational service processes: wait time, order accuracy, speed, payment, reservations. Not staff attitude.
• Cues: “slow service”, “wait time”, “efficient”, “orders messed up”.
• Positive: fast service.
• Negative: long waiting time, wrong order.
• Neutral: counter service.

STAFF (Employees, Waiters, Attitude)
• When a person is explicitly mentioned. Covers friendliness, rudeness, professionalism, helpfulness, attentiveness.
• Cues: “waiter”, “server”, “manager”, “friendly”, “rude”, “ignored”, “helpful”.
• Positive: friendly staff.
• Negative: rude waiter.
• Neutral: staff uniforms.

MISCELLANEOUS
• Overall impressions not tied to a specific aspect; general experience, overall visit, or business-level comments.
• Cues: “overall”, “experience”, “visit”, “amazing place”.
• Positive: wonderful time.
• Negative: terrible experience.
• Neutral: popular restaurant.

MENTIONED_ONLY
• Aspect is referenced but expresses no opinion, no emotion, no evaluation.
• Used only when the aspect is background info supporting another aspect.
• If ANY evaluation exists → do NOT use Mentioned_only.

MIXED EMOTIONS
• Reviewer expresses two different emotions for the same aspect category.
• Examples: “loved one dish, hated the other”, “service was helpful but frustrating”.
• Not mixed: mild variation of same polarity (“good but not great”).

-----------------------------------------
EMOTION DEFINITIONS & CUES
-----------------------------------------

ADMIRATION (Positive)
• Strong positive evaluative emotion: impressed, appreciative, respectful, elevated praise.
• Cues: “amazing”, “wow”, “impressive”, “excellent”, “stunning”, “top-notch”.
• Not admiration: weak praise (“nice”), general happiness, generic appreciation.

SATISFACTION (Positive)
• Enjoyment, comfort, pleasant surprise, happiness, relief, relaxed mood.
• Cues: “enjoyed”, “happy”, “pleasant”, “relaxing”, “surprisingly good”.
• Not satisfaction: strong admiration, gratitude, purely factual neutral.

GRATITUDE (Positive)
• Thankfulness for a specific helpful action.
• Cues: “thank you”, “grateful”, “appreciate your help”, thanking a waiter for something specific.
• Not gratitude: general appreciation of quality → admiration.

ANNOYANCE (Negative)
• Mild/moderate irritation: inconvenience, frustration, noise, crowdedness, slow service.
• Cues: “annoying”, “irritating”, “frustrating”, “bothered”, “not ideal”.
• Not annoyance: strong disappointment or disgust.

DISAPPOINTMENT (Negative)
• Unmet expectations, let-down, negative confusion.
• Cues: “disappointed”, “expected more”, “let down”, “confusing”, “sadly”, “unfortunately”.
• Not disappointment: strong anger → disgust, minor irritation → annoyance.

DISGUST (Negative)
• Strong rejection: gross food, hygiene issues, fear, unsafe environment.
• Cues: “disgusting”, “gross”, “made me sick”, “filthy”, “unsafe”, “terrifying”.
• Not disgust: mild disappointment or annoyance.

NO EMOTION (Neutral)
• Factual statements, indifference, no emotional valence.
• Cues: “okay”, “fine”, “average”, “nothing special”.
• Not neutral: weak positive “nice” → satisfaction, weak negative “not great” → disappointment.

-----------------------------------------
FINAL EMOTION SET (from PDF)
Positive: Admiration, Satisfaction, Gratitude
Negative: Disappointment, Annoyance, Disgust
Neutral: No Emotion, Mixed Emotions, Mentioned_only

-----------------------------------------
ASPECT–EMOTION PAIRS (from PDF)
ambience:
  positive: Admiration, Satisfaction
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

food:
  positive: Admiration, Satisfaction
  negative: Disgust, Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

menu:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

place:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance, Disgust
  neutral: No Emotion, Mixed Emotions, Mentioned_only

price:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

service:
  positive: Admiration, Satisfaction, Gratitude
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

staff:
  positive: Gratitude, Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

miscellaneous:
  positive: Admiration, Satisfaction
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

  """

In [ ]:
import json
import os
import requests
from dotenv import load_dotenv

# -----------------------------
# API setup
# -----------------------------
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

MODEL = "models/gemini-2.5-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"

HEADERS = {
    "Content-Type": "application/json",
    "X-goog-api-key": API_KEY
}

# -----------------------------
# Paths
# -----------------------------
IN_PATH = os.path.join(data_root, "daniel_50.jsonl")
EMOTION_JSON = os.path.join(data_root, "emotion.json")

OUT_DIR = os.path.join(results_root, "gemini-flash")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_ANNOT_PATH = os.path.join(OUT_DIR, "gemini_annotated_aspect_polarity_daniel_50.jsonl")
OUT_REASON_PATH = os.path.join(OUT_DIR, "gemini_reasons_daniel_50.jsonl")

# -----------------------------
# Load emotion taxonomy
# -----------------------------
EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))

ASPECTS = list(EMOTIONS.keys())
POLARITIES = ["positive", "negative", "neutral"]

allowed_lookup = {
    (aspect, polarity): EMOTIONS[aspect][polarity]
    for aspect in EMOTIONS
    for polarity in EMOTIONS[aspect]
}

# ----------------------------------------------------
# Gemini request
# ----------------------------------------------------
def ask_gemini(prompt):
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    r = requests.post(URL, headers=HEADERS, json=payload)
    r.raise_for_status()
    return r.json()["candidates"][0]["content"]["parts"][0]["text"].strip()

# ----------------------------------------------------
# JSON repair
# ----------------------------------------------------
def safe_json_parse(txt):
    try:
        return json.loads(txt)
    except:
        pass

    cleaned = txt.replace("```json", "").replace("```", "").strip()
    try:
        return json.loads(cleaned)
    except:
        pass

    cleaned = cleaned.replace(",]", "]").replace(",}", "}")
    try:
        return json.loads(cleaned)
    except:
        return None

# ----------------------------------------------------
# Build prompt (uses GUIDELINES from previous cell)
# ----------------------------------------------------
def build_prompt(review):
    return f"""
You are performing ABSA (Aspect-Based Sentiment & Emotion) annotation.

Follow these official annotation guidelines:
{GUIDELINES}

### Allowed aspects:
{ASPECTS}

### Allowed polarities:
{POLARITIES}

### Allowed emotions:
{json.dumps(EMOTIONS, indent=2)}

### STRICT OUTPUT FORMAT:
{{
  "triples": [
    {{"aspect": "...", "polarity": "...", "emotion": "..."}}
  ],
  "reason": "A single sentence of exactly 20 words explaining your reasoning."
}}

### RULES:
- Output MUST be ONLY valid JSON.
- No markdown, no natural-language explanation outside JSON.
- Emotion must belong to allowed list for the given (aspect, polarity).
- If no aspects appear → return `"triples": []` plus a 20-word reason.
- `reason` MUST contain exactly 20 words.

### Review:
\"""{review}\"""

Return ONLY the JSON.
"""

# ----------------------------------------------------
# Annotation logic
# ----------------------------------------------------
def annotate_full(review):
    prompt = build_prompt(review)

    parsed = None
    for _ in range(3):
        response = ask_gemini(prompt)
        parsed = safe_json_parse(response)
        if isinstance(parsed, dict) and "triples" in parsed:
            break

    if not isinstance(parsed, dict):
        print("JSON ERROR →", response)
        return [], "Reason unavailable"

    triples = parsed.get("triples", [])
    reason = parsed.get("reason", "").strip()

    # Validate triples
    final = []
    for item in triples:
        asp = item.get("aspect")
        pol = item.get("polarity")
        emo = item.get("emotion", "")

        if (asp, pol) not in allowed_lookup:
            continue

        allowed = allowed_lookup[(asp, pol)]

        # Normalize
        emo = emo.strip()
        if emo:
            emo = emo[0].upper() + emo[1:]

        if emo not in allowed:
            emo = allowed[0]

        final.append({
            "aspect": asp,
            "polarity": pol,
            "emotion": emo
        })

    return final, reason

# ----------------------------------------------------
# Load input
# ----------------------------------------------------
raw_data = [
    json.loads(line)
    for line in open(IN_PATH, "r", encoding="utf-8")
]

# ----------------------------------------------------
# Annotate all
# ----------------------------------------------------
results = []
reasons = []

for row in raw_data:
    review = row["input"]
    triples, reason = annotate_full(review)

    results.append({
        "input": review,
        "output": triples
    })

    reasons.append({
        "input": review,
        "triples": triples,
        "reason": reason
    })

# ----------------------------------------------------
# Save Files
# ----------------------------------------------------
with open(OUT_ANNOT_PATH, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

with open(OUT_REASON_PATH, "w", encoding="utf-8") as f:
    for r in reasons:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("DONE →", OUT_ANNOT_PATH)
print("REASONS →", OUT_REASON_PATH)
# import json
# import os
# import requests
# from dotenv import load_dotenv

# # -----------------------------
# # Load API key
# # -----------------------------
# load_dotenv()
# API_KEY = os.getenv("GEMINI_API_KEY")

# MODEL = "models/gemini-2.5-flash"
# URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"

# HEADERS = {
#     "Content-Type": "application/json",
#     "X-goog-api-key": API_KEY
# }

# # -----------------------------
# # Paths
# # -----------------------------
# IN_PATH = os.path.join(data_root, "daniel_50.jsonl")
# EMOTION_JSON = os.path.join(data_root, "emotion.json")

# OUT_DIR = os.path.join(results_root, "gemini-flash")  
# OUT_PATH = os.path.join(OUT_DIR, "gemini_annotated_aspect_polarity_emotions_100.jsonl")
# os.makedirs(OUT_DIR, exist_ok=True)

# # -----------------------------
# # Load emotion taxonomy
# # -----------------------------
# EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))

# ASPECTS = list(EMOTIONS.keys())
# POLARITIES = ["positive", "negative", "neutral"]

# allowed_lookup = {
#     (aspect, polarity): EMOTIONS[aspect][polarity]
#     for aspect in EMOTIONS
#     for polarity in EMOTIONS[aspect]
# }

# # -----------------------------
# # Gemini call
# # -----------------------------
# def ask_gemini(prompt):
#     payload = {"contents": [{"parts": [{"text": prompt}]}]}
#     r = requests.post(URL, headers=HEADERS, json=payload)
#     r.raise_for_status()
#     return r.json()["candidates"][0]["content"]["parts"][0]["text"].strip()


# # -----------------------------
# # JSON-safe parsing (2-stage repair)
# # -----------------------------
# def safe_json_parse(txt):
#     # Try direct
#     try:
#         return json.loads(txt)
#     except:
#         pass

#     # Try removing code fences
#     cleaned = txt.replace("```json", "").replace("```", "").strip()
#     try:
#         return json.loads(cleaned)
#     except:
#         pass

#     # Try removing trailing commas
#     cleaned = cleaned.replace(",]", "]").replace(",}", "}")
#     try:
#         return json.loads(cleaned)
#     except:
#         return None


# # -----------------------------
# # Full ABSA extraction with retry
# # -----------------------------
# def annotate_full(review):
#     prompt = f"""
# Extract aspects, polarity, and emotion.

# ### Allowed aspects:
# {ASPECTS}

# ### Allowed polarities:
# {POLARITIES}

# ### Allowed emotions:
# {json.dumps(EMOTIONS, indent=2)}

# ### Output format (STRICT):
# [
#   {{"aspect": "...", "polarity": "...", "emotion": "..."}},
#   ...
# ]

# ### Rules:
# - JSON only.
# - No explanations.
# - Emotion must be from the allowed list for that aspect/polarity.
# - If no aspects → return [].

# ### Review:
# "{review}"

# Return ONLY a JSON array.
# """

#     # Retry up to 3 times
#     for _ in range(3):
#         response = ask_gemini(prompt)
#         parsed = safe_json_parse(response)

#         if isinstance(parsed, list):
#             break

#     # Final fallback
#     if not isinstance(parsed, list):
#         print("JSON ERROR →", response)
#         return []

#     # Validate and fix emotions
#     final = []
#     for item in parsed:
#         asp = item.get("aspect")
#         pol = item.get("polarity")
#         emo = item.get("emotion", "").title()

#         if (asp, pol) not in allowed_lookup:
#             continue

#         allowed = allowed_lookup[(asp, pol)]

#         if emo not in allowed:
#             emo = allowed[0]  # fallback

#         final.append({
#             "aspect": asp,
#             "polarity": pol,
#             "emotion": emo
#         })

#     return final


# # -----------------------------
# # Load input
# # -----------------------------
# raw_data = [
#     json.loads(line)
#     for line in open(IN_PATH, "r", encoding="utf-8")
# ]




# # -----------------------------
# # Annotate all reviews
# # -----------------------------
# results = []

# for row in raw_data:
#     review = row["input"]
#     triples = annotate_full(review)

#     results.append({
#         "input": review,
#         "output": triples
#     })

# # -----------------------------
# # Save output
# # -----------------------------
# with open(OUT_PATH, "w", encoding="utf-8") as f:
#     for r in results:
#         f.write(json.dumps(r, ensure_ascii=False) + "\n")

# print("DONE →", OUT_PATH)